In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

# 밀키트 전체 인기 검색어 가져오기

In [8]:
#인기 검색어 내용 추출
def get_rank_list(driver):
    trend_list = []
    rank_texts = driver.find_elements(by=By.CSS_SELECTOR, value='div.rank_top1000_scroll > ul')
    for rank_text in rank_texts:
        rank_text = rank_text.text
        rank_text = rank_text.split('\n')
        trend_list.extend(rank_text[1::2])
    return trend_list


#인기 검색어 1-40위까지 추출
def get_1_to_40(driver, TIMEOUT):
    trend_1_to_40 = []
    rank_text = get_rank_list(driver) #(1-20)
    trend_1_to_40.extend(rank_text)

    next_btn = driver.find_element(By.CSS_SELECTOR, 'div.top1000_btn_area > div > a.btn_page_next') #(21-40)
    next_btn.click()
    time.sleep(TIMEOUT)
    rank_text = get_rank_list(driver)
    trend_1_to_40.extend(rank_text)
    return trend_1_to_40

In [9]:
url='https://datalab.naver.com/shoppingInsight/sCategory.naver?cid=50014240'
TIMEOUT=3

try:
    #service = Service(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(TIMEOUT)
    
    #전체 밀키트 인기검색어
    all_trend_list = []
    all_trend_list.append(get_1_to_40(driver, TIMEOUT))

    #세부카테고리의 인기 검색어 : '찌개/국','면/파스타','구이','볶음/튀김','조림/찜'
    for idx in range(2,7):
        driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/span').click() #select_btn
        time.sleep(TIMEOUT)
        driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li['+str(idx)+']/a').click() #option
        time.sleep(TIMEOUT)
        driver.find_element(By.CSS_SELECTOR, '#content > div.section_instie_area.space_top > div > div.section.insite_inquiry > div > a').click() #btn_submit
        time.sleep(TIMEOUT)
        all_trend_list.append(get_1_to_40(driver, TIMEOUT))
        
except Exception:
    raise

finally:
    if driver is not None:
        driver.quit()

In [10]:
print(all_trend_list)

[['밀키트', '떡볶이', '낙곱새', '쌀떡볶이', '알탕', '캠핑밀키트', '해신탕', '국물떡볶이', '스테이크밀키트', '충무로한우곱창전골', '동태밥상', '만두전골', '애슐리밀키트', '떡볶이밀키트', '사과떡볶이', '즉석떡볶이', '곱창전골', '자미곱', '더원푸드시래기곱창전골밀키트', '곱창전골밀키트', '크리스마스밀키트', '감바스밀키트', '마라탕밀키트', '수제비밀키트', '동남아식밀키트탄탄면', '설렁탕', '더원푸드시래기전골밀키트', '홈파티밀키트', '닭갈비밀키트', '공항칼국수', '홈파티음식', '파스타밀키트', '보일링크랩', '군고기밀키트', '목살밀키트', '소고기버섯칼국수', '프레시지', '찜닭밀키트', '밀푀유나베밀키트', '우거지감자탕밀키트순살해장국우거지감자탕밀키트'], ['밀키트', '낙곱새', '알탕', '캠핑밀키트', '해신탕', '충무로한우곱창전골', '동태밥상', '만두전골', '곱창전골', '더원푸드시래기곱창전골밀키트', '곱창전골밀키트', '마라탕밀키트', '설렁탕', '더원푸드시래기전골밀키트', '우거지감자탕밀키트순살해장국우거지감자탕밀키트', '밀푀유나베밀키트', '마라탕', '곱창전골한우소곱창전골', '프리미엄부대찌개밀키트김치2인3인캠핑요리쿠킹박스술안주', '우거지감자탕밀키트순살해장국', '부대찌개밀키트', '소고기된장밀키트당일조리', '갈비탕밀키트', '해신탕활전복4인분', '밀푀유나베', '캠핑용밀키트', '국밥밀키트', '부대찌개밀키트2인분', '돼지국밥밀키트', '해물전골캠핑밀키트', '해물탕밀키트', '국밥', '돼지짜글이', '간편하게즐기는곱창전골', '밀키트만두전골', '속초홍게라면캠핑요리', '호랑이굴대창전골', '샤브샤브밀키트', '낙곱새밀키트', '부산맛집밀키트(1회용냄비선물)당감댁곱도리탕닭볶음탕닭도리탕곱창안주캠핑용'], ['밀키트', '수제비밀키트', '동남아식밀키트탄탄면', '공항칼국수', '파스타밀키트', '소고기버섯칼국수', '애슐리밀키트', '라자냐밀키트', '라자냐'

In [20]:
#데이터프레임으로 정리
detailed_category = ['전체', '찌개/국','면/파스타','구이','볶음/튀김','조림/찜']
trend_df = pd.DataFrame(all_trend_list).T
trend_df.columns = detailed_category
trend_df.to_csv('네이버데이터랩_인기검색어.csv')
trend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   전체      40 non-null     object
 1   찌개/국    40 non-null     object
 2   면/파스타   40 non-null     object
 3   구이      40 non-null     object
 4   볶음/튀김   40 non-null     object
 5   조림/찜    40 non-null     object
dtypes: object(6)
memory usage: 2.0+ KB


In [19]:
trend_df.head()

,전체,찌개/국,면/파스타,구이,볶음/튀김,조림/찜
0,밀키트,밀키트,밀키트,밀키트,밀키트,밀키트
1,떡볶이,낙곱새,수제비밀키트,스테이크밀키트,감바스밀키트,보일링크랩
2,낙곱새,알탕,동남아식밀키트탄탄면,자미곱,닭갈비밀키트,찜닭밀키트
3,쌀떡볶이,캠핑밀키트,공항칼국수,군고기밀키트,애슐리밀키트,갈비찜오리지널맛
4,알탕,해신탕,파스타밀키트,목살밀키트,감바스,우족찜
